# Some Boilerplate stuff

In [1]:
# You may prefer to upload the data to your google drive and mount your google drive to this colab, 
# because the data will be erased if you stop using this colab for a while.
# Uncomment the code below to do so. After mounting, navigate to the appropriate folder, right click, and "copy path".
# Assign DATA_DIR global variable to that path.
# For more mounting instructions: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# If imported from google drive, config for your file directory. Mine is 'lm_data'.
DATA_DIR = "./drive/MyDrive/nlp-final-project/data"

# the goal is that DATA_DIR points to where the training/validation/test data is. 

In [3]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 3.8 MB 48.6 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 212 kB 47.4 MB/s 
     |████████████████████████████████| 134 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 596 kB 29.6 MB/s 
     |████████████████████████████████| 6.5 MB 38.8 MB/s 
     |████████████████████████████████| 271 kB 50.5 MB/s 
     |████████████████████████████████| 144 kB 40.7 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=25813ec41dbe8b6218c6fc58e0d2a976a836acfe3d7d23c0841fc33e22a20f68
  Stored in directory: /root/.cache/pip/whe

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 2s (970 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [6]:
import transformers

print(transformers.__version__)

4.17.0


In [7]:
model_checkpoint = "bert-base-uncased"

batch_size = 16

# Loading Data

In [35]:
import csv
import ast
from datasets import Dataset, load_metric, DatasetDict
import numpy as np
import random

# Get raw datasets
'''
1.1 VUA
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a list of labels: 
    a list of pos: 
'''
inputs = []
with open(f'{DATA_DIR}/TROFI/TroFi_formatted_all3737.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        input = {}
        input["tokens"] = line[1].split()
        index = int(line[2])
        label = int(line[3])
        labels = [0] * len(input["tokens"])
        labels[index] = label
        input["labels"] = labels
        inputs.append(input)
random.shuffle(inputs)
train_val, test = np.split(np.array(inputs), [int(len(inputs)/2)])
train, val = np.split(np.array(train_val), [int(len(train_val)/2)])
print(train.shape)
print(val.shape)
print(test.shape)

raw_train_vua = {}
raw_train_vua["tokens"] = []
raw_train_vua["tags"] = []
for item in train:
    raw_train_vua["tokens"].append(item["tokens"])
    raw_train_vua["tags"].append(item["labels"])

raw_val_vua = {}
raw_val_vua["tokens"] = []
raw_val_vua["tags"] = []
for item in val:
    raw_val_vua["tokens"].append(item["tokens"])
    raw_val_vua["tags"].append(item["labels"])

raw_test_vua = {}
raw_test_vua["tokens"] = []
raw_test_vua["tags"] = []
for item in test:
    raw_test_vua["tokens"].append(item["tokens"])
    raw_test_vua["tags"].append(item["labels"])

dataset_dict = {}
dataset_dict["train"] = Dataset.from_dict(raw_train_vua)
dataset_dict["test"] = Dataset.from_dict(raw_test_vua)
dataset_dict["validation"] = Dataset.from_dict(raw_val_vua)

datasets = DatasetDict(dataset_dict)

(934,)
(934,)
(1869,)


In [37]:
print(datasets["train"]["tokens"][0])
print(datasets["train"]["tags"][0])

['His', 'boss', 'saw', 'it', 'as', 'insubordination', 'and', 'kicked', 'Mr.', 'Arnold', 'out']
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [38]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

In [39]:
label_all_tokens = True

In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [40]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Train model

In [41]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=2)


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

In [42]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"bert-lets-go",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [44]:
metric = load_metric("seqeval")

In [45]:
import numpy as np
label_list = ["O", "I"]
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [46]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 934
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1770


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.040736,1.000000,0.007143,0.014184,0.985022
2,No log,0.030773,0.570025,0.552381,0.561064,0.987080


The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 934
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 934
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 934
 

In [25]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 934
  Batch size = 16


{'epoch': 30.0,
 'eval_accuracy': 0.9889384183589236,
 'eval_f1': 0.5725593667546175,
 'eval_loss': 0.07984669506549835,
 'eval_precision': 0.6326530612244898,
 'eval_recall': 0.5228915662650603,
 'eval_runtime': 8.7326,
 'eval_samples_per_second': 106.955,
 'eval_steps_per_second': 6.756}

# Evaluate model

In [26]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

label_list = ["O", "I"]
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1869
  Batch size = 16


{'_': {'precision': 0.593103448275862, 'recall': 0.5422446406052963, 'f1': 0.5665349143610012, 'number': 793}, 'overall_precision': 0.593103448275862, 'overall_recall': 0.5422446406052963, 'overall_f1': 0.5665349143610012, 'overall_accuracy': 0.9884886865888252}


In [27]:
print(predictions)
print(labels)

result = []
words = []
for ids in tokenized_datasets["test"]["input_ids"]:
    words.append(tokenizer.convert_ids_to_tokens(ids))
for i in range(len(true_labels)):
    result_entry_list = []
    for j in range(len(true_predictions[i])):
        result_entry_list.append(f"{words[i][j+1]}({true_predictions[i][j]} {true_labels[i][j]})")
    result_entry = " ".join(result_entry_list)
    result_entry = f"{result_entry}\n\n"
    result.append(result_entry)

f = open(f'{DATA_DIR}/predictions/trofi_seq_test_predictions_BERTsequence_vua.txt', 'w+')
f.writelines(result)
f.close()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[-100    0    0 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]
 ...
 [-100    0    0 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]
 [-100    0    0 ... -100 -100 -100]]


# Detokenization

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

label_list = ["O", "I"]
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

In [28]:
def detokenize(examples, predictions, labels):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    tokens = examples["tokens"]
    re_preds = []
    re_labels = []
    for i in range(len(examples[f"tags"])):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        re_pred = ["O"] * len(tokens[i])
        re_label = ["O"] * len(tokens[i])
        for j, word_idx in enumerate(word_ids):
            if j >= len(labels[i]):
                break
            if word_idx is None:
                continue
            else:
                if labels[i][j] == 'I':
                    re_label[word_idx] = "I"
                if predictions[i][j] == 'I':
                    re_pred[word_idx] = "I"
        re_labels.append(re_label)
        re_preds.append(re_pred)
    return (re_preds, re_labels)

In [30]:
re_preds, re_labels = detokenize(datasets["test"], true_predictions, true_labels)

results = metric.compute(predictions=re_preds, references=re_labels)
print(results)

{'_': {'precision': 0.5983379501385041, 'recall': 0.544766708701135, 'f1': 0.5702970297029704, 'number': 793}, 'overall_precision': 0.5983379501385041, 'overall_recall': 0.544766708701135, 'overall_f1': 0.5702970297029704, 'overall_accuracy': 0.987222721822542}
